In [ ]:
一 使用 docker 运行软件

In [ ]:
# 运行数据库 redis
docker run -d -p 6379:6379 --name redis redis:latest

In [ ]:
# 参数含义
-d 表示 detach 或者 daemon，即后台运行；不加 -d 会阻断，并在终端直接运行
-p 表示 port 端口映射，即主机端口 6379 映射到容器端口 6379
--name 表示容器（container）的名字，可以自定义为 123
redis：latest 表示 镜像（imagine）名和对应的 tag（版本号）

In [ ]:
二 使用 docker 打包自己的镜像

In [ ]:
step1 创建 dockerfile

在项目的根目录下，创建 dockerfile
vim Dockerfile

In [ ]:
dockerfile 的语法结构
# 直径基础环境，比如 python
FROM python：3.9.21

# MAINTAINER XXX（项目维护人）

# 设置工作目录（软件包的根目录）
WORKDIR /app

# 复制项目文件到容器中
COPY ./app

# 设置清华源并安装依赖包
# RUN命令可以有多个，但是可以用 && 连接多个命令来减少层级
# 例如 RUN npm install && cd /app && mkdir logs
RUN pip install --no-cache-dir --index-url https://pypi.tuna.tsinghua.edu.cn/simple -r requirements.txt

# 设置环境变量（可选）
ENV DB_HOST=db_host \
    DB_PORT=3306 \
    DB_DATABASE=mydatabase \ 
    DB_USER=myuser \
    DB_PASSWORD=mypassword

# 暴露端口（可选）
EXPOSE ${SERVER_PORT}

# 启用应用文件
# CMD 指令只能一个，是容器启动后执行的命令，算是程序入口
# 如果还需要执行其他命令可以使用&&连接，也可以写成一个shell脚本
# 例如 CMD cd/app && ./start.sh
CMD python test.py

In [ ]:
step2 创建 requierments.txt 文件

In [ ]:
在项目的根目录下创建名为requirements.txt 文件，列出所有必要的 python 依赖包。
pycharm可以自动生成依赖。
方法一：项目根目录下，在终端直接输入）
pip freeze > requirements.txt
方法二：通过 pipreqs 生成
# 安装 pipreqs
pip install pipreqs
# 在根目录下创建
pipreqs ./

In [ ]:
两种方法都可以创建 requirements.txt 但存在明显区别：
方法一创建的包含所有 packages 安装包，更细粒度；方法二创建的主要是pip 安装过的安装包
建议首先使用方法二，不行的话再使用方法一
使用方法二创建的 requirements.txt 如下所是：
datasets==3.1.0
matplotlib==3.7.5
numpy==1.24.3
numpy==1.24.4
Pillow==11.1.0
torch==2.4.0
torchvision==0.19.0
tqdm==4.67.1
transformers==4.49.0
wandb==0.19.7

In [ ]:
step3 创建环境变量

In [ ]:
# 在项目的根目录下创建一个名为 .env 的文件（根据实际情况修改，也可以不创建）
DB_HOST=db_host
DB_PORT=3306
DB_DATABASE=mydatabase
DB_USER=myuser
DB_PASSWORD=mypassword

In [ ]:
# 在 python 程序中配置 config.py，使用 os.environ.get 方法读取环境变量
# 如果环境变量不存在，可以提供一个默认值
import os

# config.py
VERSION = os.environ.get('VERSION', 'v1.0.0')
DB_HOST = os.environ.get('DB_HOST', '127.0.0.1')
DB_PORT = os.environ.get('DB_PORT', '3306')
DB_DATABASE = os.environ.get('DB_DATABASE', 'test_db')
DB_USER = os.environ.get('DB_USER', 'root')
DB_PASSWORD = os.environ.get('DB_PASSWORD', '123456')

In [ ]:
对于创建的 .env 文件，需要在python项目中进行加载处理
# 使用 python-dotenv 库加载环境变量
from flask import Flask, request, jsonify
from dotenv import load_dotenv
import os
# 加载 .env 文件
load_dotenv()

app = Flask(__name__)

@app.route('/')
def hello():
    return "Hello, World!"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=int(os.getenv('SERVER_PORT', 8028)))

In [ ]:
step4 构建 docker 镜像
# 使用 terminal 终端导航到 dockerfile 的目录，构建 DOCKER 镜像
docker build -t my-rdbackend .

In [ ]:
-build 表示构建dockerfile
-t 表示 tag，即镜像名
- my-rdbackend 是镜像名
-. 表示当前目录下的所有文件 

In [ ]:
step5 运行docker容器

docker run -d -p 5001:5000 --env-file .env my-rdbackend

-p 5001:5000 参数映射了主机的 5001 端口到容器的 5000 端口
–env-file .env 参数指定了一个环境变量文件，该文件包含了所有需要传递给容器的环境变量值。

step6 查看容器有没有运行成功
docker ps

In [ ]:
Dockerfile 常用指令集

FROM 这个镜像的妈妈是谁？（指定基础镜像）
MAINTAINER 告诉别人，谁负责养它？（指定维护者信息，可以没有）LABLE key=values
RUN 你想让它干啥（在命令前面加上RUN即可）
ADD/COPY 给它点创业资金（COPY文件，会自动解压）
WORKDIR 我是cd,今天刚化了妆（设置当前工作目录）
VOLUME 给它一个存放行李的地方（设置卷，挂载主机目录）
EXPOSE 它要打开的门是啥（指定对外的端口）
CMD 奔跑吧，兄弟！（指定容器启动后的要干的事情）
ENTRYPOINT  容器启动命名
ENV 环境变量
USER 切换用户

In [ ]:
Docker build 语法
# docker build 指令用于根据给定的 Dockerfile 构建 Docker 镜像

# docker build 语法
docker build [OPTIONS]<PATH|URL->

# 常用选项说明
--build-arg，设置构建时的变量
--no-cache，默认false。设置该选项，将不使用Build Cache构建镜像
--pull，默认false。设置该选项，总是尝试pull镜像的最新版本
--compress，默认false。设置该选项，将使用gzip压缩构建的上下文
--disable-content-trust，默认true。设置该选项，将对镜像进行验证
--file, -f，Dockerfile的完整路径，默认值为‘PATH/Dockerfile’
--isolation，默认--isolation="default"，即Linux命名空间；其他还有process或hyperv
--label，为生成的镜像设置metadata
--squash，默认false。设置该选项，将新构建出的多个层压缩为一个新层，但是将无法在多个镜像之间共享新层；设置该选项，实际上是创建了新image，同时保留原有image。
--tag, -t，镜像的名字及tag，通常name:tag或者name格式；可以在一次构建中为一个镜像设置多个tag
--network，默认default。设置该选项，Set the networking mode for the RUN instructions during build
--quiet, -q ，默认false。设置该选项，Suppress the build output and print image ID on success
--force-rm，默认false。设置该选项，总是删除掉中间环节的容器
--rm，默认--rm=true，即整个构建过程成功后删除中间环节的容器 

# 示例
docker build -t soso/bbauto:v2.1 .

-docker build  是docker创建镜像的命令 
-t 是标识新建的镜像属于 soso的 bbauto镜像 
：v2.1 是tag 
"."是用来指明使用的Dockerfile文件当前目录的（也可以是根绝对路径）